In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.7 MB/s eta 0:00:00


In [ ]:
import hashlib
import json
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode

# Function to derive AES key from password
def derive_key(password, use_sha256=True):
    if use_sha256:
        return hashlib.sha256(password.encode()).digest()
    else:
        return hashlib.md5(password.encode()).digest()

# Function to encrypt message
def encrypt_message(username, password, message, use_sha256=True):
    key = derive_key(password, use_sha256)
    iv = os.urandom(16)  # Random IV for CBC mode
    cipher = AES.new(key, AES.MODE_CBC, iv)

    encrypted_message = cipher.encrypt(pad(message.encode(), AES.block_size))
    hash_value = hashlib.sha256(encrypted_message).hexdigest()

    data = {
        "username": username,
        "iv": b64encode(iv).decode(),
        "encrypted_message": b64encode(encrypted_message).decode(),
        "hash": hash_value
    }

    # Store encrypted data in JSON file
    with open(f"/content/drive/My Drive/{username}_encrypted.json", "w") as file:
        json.dump(data, file)

    print("[INFO] Encrypted message stored successfully.")

# Function to decrypt message
def decrypt_message(username, password, use_sha256=True):
    try:
        # Load encrypted data
        with open(f"/content/drive/My Drive/{username}_encrypted.json", "r") as file:
            data = json.load(file)

        key = derive_key(password, use_sha256)
        iv = b64decode(data["iv"])
        encrypted_message = b64decode(data["encrypted_message"])
        stored_hash = data["hash"]

        # Verify integrity
        if hashlib.sha256(encrypted_message).hexdigest() != stored_hash:
            print("[ERROR] Integrity check failed! Possible tampering detected.")
            return

        # Decrypt message
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted_message = unpad(cipher.decrypt(encrypted_message), AES.block_size).decode()

        print("[INFO] Hash verified! Decrypting message...")
        print("Decrypted message:", decrypted_message)

    except FileNotFoundError:
        print("[ERROR] No encrypted message found for user.")
    except Exception as e:
        print("[ERROR] An error occurred:", e)

# User interaction
action = input("Do you want to (E)ncrypt or (D)ecrypt a message? ").strip().upper()
username = input("Enter your username: ").strip()
password = input("Enter your password: ").strip()
use_sha256 = input("Use SHA-256 (Y) or MD5 (N)? ").strip().upper() == "Y"

if action == "E":
    message = input("Enter message to encrypt: ")
    encrypt_message(username, password, message, use_sha256)
elif action == "D":
    decrypt_message(username, password, use_sha256)
else:
    print("[ERROR] Invalid choice.")

Do you want to (E)ncrypt or (D)ecrypt a message? d
Enter your username: galib
Enter your password: password
Use SHA-256 (Y) or MD5 (N)? n
[INFO] Hash verified! Decrypting message...
Decrypted message: new message


In [ ]:
import hashlib

# Function to compute SHA-256 hash of a file
def compute_sha256(file_path):
    with open(file_path, 'rb') as f:
        file_data = f.read()
    return hashlib.sha256(file_data).hexdigest()

# Load private key and compute hash
private_key_path = "/content/drive/My Drive/Privacy PA1/Keys-Cipher Text/private_key.pem"
computed_hash = compute_sha256(private_key_path)

# Provided hash options (update these based on your assignment)
provided_hashes = {
    "A": "1a4c8b9d847b3e2fa2d5f9d31c8e5f8b7c91a5d0f4e5b2f7d8c6e2a9b4d5e1c3",
    "B": "9e427f6bea8af1fc9d2d332312338cf538759ebe5f71843af205c18d726623f9",
    "C": "3f4b5c2d9a8e7f1c5d3b2a9c6e1d4f7b8a2c5e9f1d0b3a7c8e6d2f5b9a4c3e1d",
    "D": "0d09a513353e632b068a1a49e6ecc0b2c753ccc1c95cb1751745ba576d1396c8"
}

# Check match
for key, hash_value in provided_hashes.items():
    if computed_hash == hash_value:
        print(f"[INFO] Hash matches option {key}!")
        break
else:
    print("[ERROR] No matching hash found.")


[INFO] Hash matches option D!


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import base64

# Load Alice's private key
private_key_path = "/content/drive/My Drive/Privacy PA1/Keys-Cipher Text/private_key.pem"
with open(private_key_path, "rb") as f:
    private_key = RSA.import_key(f.read())

# Initialize RSA decryption using PKCS1_OAEP
cipher_rsa = PKCS1_OAEP.new(private_key)

# Load and decode the encrypted message
encrypted_message_path = "/content/drive/My Drive/Privacy PA1/Keys-Cipher Text/encrypted_message.txt"
with open(encrypted_message_path, "r") as f:
    encrypted_message_base64 = f.read().strip()

# Decode from Base64
encrypted_message_bytes = base64.b64decode(encrypted_message_base64)

# Decrypt the message
try:
    decrypted_message = cipher_rsa.decrypt(encrypted_message_bytes).decode()
    print("[INFO] Decrypted message:", decrypted_message)
except Exception as e:
    print("[ERROR] Decryption failed:", str(e))


[INFO] Decrypted message: You can do anything, no one can judge you, hard work and discipline is all you need
